In [ ]:
## Install dependencies
!apt install coinor-libipopt-dev
!pip install ipopt
!pip install cvxpylayers
!pip install gurobipy

In [ ]:
import numpy as onp
import matplotlib.pylab as plt
from functools import partial
import time
import torch
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import torch 
import torch.nn as nn

In [ ]:
## parameters

T = 2.0 # prediction horizon of MPC
N = 10  # number of prediction steps of MPC
ts = T/N  # sampling interval
min_v = 0.1 # minimum value of the first and third control inputs
u_lb = [min_v, -1.0, min_v, -1.0] # lower bounds of the control inputs
u_ub = [1.0,  1.0, 1.0,  1.0] # upper bounds of the control inputs
u_r_lb = [min_v, -1.0, min_v, -1.0, 0] # lower bounds of the control inputs + Chebyshev radius (the last one)
u_r_ub = [1.0,  1.0, 1.0,  1.0, 10] # upper bounds of the control inputs + Chebyshev radius (the last one)
Ds = 0.5 # safe distance
v  = 0.1 # parameter used in true CBF
w  = 1.0 # parameter used in true CBF
sf = 1
DPChorizon = 50
n_in, n_h, n_hh, n_hhh, n_out = 6, 50, 100, 50, 4 # parameters for NN architecture
learningRate = 0.005 # NN learning rate
epochNumber = 1000 # NN training epochs
start_angle = 3.1415926
angle_cost_coefficient = 1.0
input_cost_coefficient = 0.001
regCoefficient = 0.001 # for LP regularizer
goal_angle = 3.1415926
inputBounds = [1., -min_v, 1., 1., 1., -min_v, 1., 1.] # input bounds vector used in gauge map
# input bounds matrix used in gauge map
inputBoundsMatrix = [torch.tensor([[1.0], [0.0], [0.0], [0.0]]), 
                     torch.tensor([[-1.0], [0.0], [0.0], [0.0]]),
                     torch.tensor([[0.0], [1.0], [0.0], [0.0]]),
                     torch.tensor([[0.0], [-1.0], [0.0], [0.0]]),
                     torch.tensor([[0.0], [0.0], [1.0], [0.0]]),
                     torch.tensor([[0.0], [0.0], [-1.0], [0.0]]),
                     torch.tensor([[0.0], [0.0], [0.0], [1.0]]),
                     torch.tensor([[0.0], [0.0], [0.0], [-1.0]])]

In [ ]:
## Basic functions

import time

def ODE_RHS(x,u):
# RHS of the aircraft dynamics
    xdot = [u[0]*onp.cos(x[2]), u[0]*onp.sin(x[2]), u[1], u[2]*onp.cos(x[5]), u[2]*onp.sin(x[5]), u[3] ]
    return onp.array(xdot)

def dynamics_RK4(x, u, T, N):
# RK4 integrator
    M = 4 # RK4 steps per interval
    DT = T/N/M
    X = onp.array(x)
    U = onp.array(u)
    for j in range(M):
        k1 = ODE_RHS(X, U)
        k2 = ODE_RHS(X + DT/2 * k1, U)
        k3 = ODE_RHS(X + DT/2 * k2, U)
        k4 = ODE_RHS(X + DT * k3, U)
        X=X+DT/6*(k1 +2*k2 +2*k3 +k4)
    return list(X)

def my_dynamics(x, u):
    dt = 0.1
    #print(x)
    #print(u)
    dotx = torch.tensor([u[0] * torch.cos(torch.tensor(x[2])), u[0] * torch.sin(torch.tensor(x[2])), u[1], u[2] * torch.cos(torch.tensor(x[5])), u[2] * torch.sin(torch.tensor(x[5])), u[3]])
    next_x = torch.tensor(x) + dt * dotx
    return next_x

def ZCBF(x):
# The constructive CBF in [Squires et al]
    A1 = db(x)**2 + dc(x)**2 + 2*(v/w)**2 - 2*((v/w)**2)*onp.cos(x[2]-x[5])
    A2 = compute_A2(x)
    h  = A1 - A2 - Ds**2
    return h

def num_grad(x):
# Numerically compute the gradient of CBF h(x) w.r.t. x
    eps_grad = 1e-10;
    dhdx = [];
    for i in range(x.size):
        Eps = 0*x;
        Eps[i] = eps_grad;
        dhdx.append((ZCBF(x+Eps)-ZCBF(x-Eps))/(2*eps_grad))
    return dhdx # return a list


def ZCBF_grad(x):
# The constructive CBF in [Squires et al]
    A1 = db_grad(x)**2 + dc_grad(x)**2 + 2*(v/w)**2 - 2*((v/w)**2)*torch.cos(x[:, 2]-x[:, 5])
    A2 = compute_A2_grad(x)
    h  = A1 - A2 - Ds**2
    return h

def num_grad_grad(x):
# Numerically compute the gradient of CBF h(x) w.r.t. x
    eps_grad = 1e-10;
    dhdx = [];
    for i in range(x.shape[1]):
        Eps = 0*x;
        Eps[:, i] = eps_grad;
        dhdx.append((ZCBF_grad(x+Eps)-ZCBF_grad(x-Eps))/(2*eps_grad))
    return dhdx # return a list


def safe_closed_loop(x_init, x_goal, N_sim, plot):
# Closed-loop Simulations with CBF+MPC
    Xcl = []
    Ucl = []
    Xcl.append(onp.array(x_init))
    xt = x_init

    for t in range(N_sim):
      u_opt = CFTOC_NMPC(xt, x_goal, T, N) # Solve the NMPC problem at the current time
      u_mod = CBF_QP_Gurobi(xt, u_opt) # Solve the CBF-QP
      xt = dynamics_RK4(xt, u_mod, T, N) # Evolve the system
      Xcl.append(onp.array(xt))
      Ucl.append(onp.array(u_mod))
    
    Xcl = onp.array(Xcl)
    Ucl = onp.array(Ucl)
    cost = trajCost(Xcl, Ucl)
    print("MPC traj cost is: ", cost)
    print("---------------------------")
    if plot:
      distValuePlot(Xcl)
      tgrid = [ts*k for k in range(N_sim)]
      plt.figure(figsize=(10,5))
      plt.clf()
      plt.plot(Xcl[:,0], Xcl[:,1], '-')
      plt.plot(Xcl[:,3], Xcl[:,4], '-')

      plt.xlabel('$p_x$')
      plt.ylabel('$p_y$')
      plt.legend(['agent a','agent b'])
      plt.xlim([-3,3])
      plt.ylim([-2,2])
      plt.axis('equal')
      plt.grid()
      #images_dir = '/content/drive/My Drive/Colab Notebooks/figures'
      #plt.savefig(f"{images_dir}/trajectory-MPC.png")
      plt.figure()
      plt.clf()
      plt.step(tgrid, Ucl[:,0], '-.')
      plt.step(tgrid, Ucl[:,1], '-.')
      plt.step(tgrid, Ucl[:,2], '-.')
      plt.step(tgrid, Ucl[:,3], '-.')
      plt.xlabel('t')
      plt.legend(['u1','u2','u3','u4'])
      plt.grid()
      #plt.savefig(f"{images_dir}/input-MPC.png")
      plt.show()
    return Xcl, Ucl
  

def plot_closed_loop(Xcl, Ucl, timesteps):
# Plot closed-loop state and input trajectories
    for t in timesteps:
      tgrid = [ts*k for k in range(t)]
      plt.figure()
      plt.clf()
      plt.subplot(1,2,1)
      plt.plot(Xcl[:t,0], Xcl[:t,1], '-')
      plt.plot(Xcl[:t,3], Xcl[:t,4], '-')
      plt.xlabel('x')
      plt.legend(['agent a','agent b'])
      plt.grid()
      plt.xlim([-3,3])
      plt.ylim([-2,2])
      plt.subplot(1,2,2)
      plt.step(tgrid, Ucl[:t,0], '-.')
      plt.step(tgrid, Ucl[:t,1], '-.')
      plt.step(tgrid, Ucl[:t,2], '-.')
      plt.step(tgrid, Ucl[:t,3], '-.')
      plt.xlabel('t')
      plt.legend(['u1','u2','u3','u4'])
      plt.grid()
      plt.show()


def chebyshevGurobiTrain(x):
    #start_time = time.time()
    u_r_lb_new = [min_v, -1.0, min_v, -1.0, 0, 0] # lower bounds of the control inputs + Chebyshev radius (the last one)
    u_r_ub_new = [1.0,  1.0, 1.0,  1.0, 10, 10] 
    m = gp.Model("LP")
    m.setParam('OutputFlag', 0)
    u = m.addMVar(shape=(1,6), lb=u_r_lb_new, ub=u_r_ub_new, name='u')
    unsafe_penalty = 1000.0
    regular_unique = 0.001
    hx   = ZCBF_grad(x)
    dhdx = num_grad_grad(x)
    cbfF = (- dhdx[0]*torch.cos(x[0, 2]) - dhdx[1]*torch.sin(x[0, 2])).reshape((1,1))
    cbfF = torch.cat((cbfF, (- dhdx[2]).reshape((1,1))), 0)
    cbfF = torch.cat((cbfF, (- dhdx[3]*torch.cos(x[0, 5]) - dhdx[4]*torch.sin(x[0, 5])).reshape((1,1))), 0)
    cbfF = torch.cat((cbfF, (- dhdx[5]).reshape((1,1))), 0)
    cbfg = hx ** 3
    #chebyPara = (dhdx[0]*onp.cos(x[2]) - dhdx[1]*onp.sin(x[2]))**2 + dhdx[2]**2 + (dhdx[3]*onp.cos(x[5]) - dhdx[4]*onp.sin(x[5]))**2 + dhdx[5] ** 2
    chebyPara = (dhdx[0]*torch.cos(x[0, 2])) ** 2 + (dhdx[1]*torch.sin(x[0, 2]))**2 + dhdx[2]**2 + (dhdx[3]*torch.cos(x[0, 5])) ** 2 + (dhdx[4]*torch.sin(x[0, 5]))**2 + dhdx[5] ** 2
    m.addConstr(dhdx[0]*(u[0,0]*torch.cos(x[0, 2])) + dhdx[1]*(u[0,0]*torch.sin(x[0, 2])) + dhdx[2]*u[0,1] + dhdx[3]*(u[0,2]*torch.cos(x[0, 5])) + dhdx[4]*(u[0,2]*torch.sin(x[0, 5])) + dhdx[5]*u[0,3] + hx**3 - u[0,4] * chebyPara >= 0)

    m.addConstr(-u[0,4]-min_v+u[0,0] >= 0.)
    m.addConstr(1.- u[0,0] - u[0,4] >= 0.)
    m.addConstr(1.- u[0,2] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]-min_v+u[0,2] >= 0.)
    m.addConstr(1.- u[0,1] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]+1.0+u[0,1] >= 0.)
    m.addConstr(1.- u[0,3] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]+1.0+u[0,3] >= 0.)
    m.setObjective(-u[0, 4] + regular_unique * (u[0,0] ** 2 + u[0,1] ** 2 + u[0,2] ** 2+u[0,3] ** 2), GRB.MINIMIZE)# + unsafe_penalty * u[0, 5]
    m.optimize()
    center = u.getAttr("x")[0,0:4]
    #print("center & cbfF & cbfg & chebypara & radius", center, cbfF, cbfg, chebyPara, u.getAttr("x")[0,4])
    #print("---------------------------------------------")
    #cbfValue = dhdx[0]*(modifiedU[0]*onp.cos(x[2])) + dhdx[1]*(modifiedU[0]*onp.sin(x[2])) + dhdx[2]*modifiedU[1] + dhdx[3]*(modifiedU[2]*onp.cos(x[5])) + dhdx[4]*(modifiedU[2]*onp.sin(x[5])) + dhdx[5]*modifiedU[3] + hx**3
    return torch.tensor(center), cbfF, cbfg


def chebyshevGurobi(x):
    #start_time = time.time()
    m = gp.Model("LP")
    m.setParam('OutputFlag', 0)
    #m.params.NonConvex = 2
    #u = m.addMVar(shape=(1,5), name='u') # u ([0:4]) and R ([4])
    u = m.addMVar(shape=(1,5), lb=u_r_lb, ub=u_r_ub, name='u')
    x  = onp.array(x) # Current states
    hx   = ZCBF(x)
    dhdx = num_grad(x)
    cbfF = torch.tensor([[- dhdx[0]*onp.cos(x[2]) - dhdx[1]*onp.sin(x[2])], [- dhdx[2]], [- dhdx[3]*onp.cos(x[5]) - dhdx[4]*onp.sin(x[5])], [- dhdx[5]]])
    cbfg = torch.tensor(hx ** 3)
    #chebyPara = (dhdx[0]*onp.cos(x[2]) - dhdx[1]*onp.sin(x[2]))**2 + dhdx[2]**2 + (dhdx[3]*onp.cos(x[5]) - dhdx[4]*onp.sin(x[5]))**2 + dhdx[5] ** 2
    chebyPara = (dhdx[0]*onp.cos(x[2])) ** 2 + (dhdx[1]*onp.sin(x[2]))**2 + dhdx[2]**2 + (dhdx[3]*onp.cos(x[5])) ** 2 + (dhdx[4]*onp.sin(x[5]))**2 + dhdx[5] ** 2
    m.addConstr(dhdx[0]*(u[0,0]*onp.cos(x[2])) + dhdx[1]*(u[0,0]*onp.sin(x[2])) + dhdx[2]*u[0,1] + dhdx[3]*(u[0,2]*onp.cos(x[5])) + dhdx[4]*(u[0,2]*onp.sin(x[5])) + dhdx[5]*u[0,3] + hx**3 - u[0,4] * chebyPara >= 0)

    m.addConstr(-u[0,4]-min_v+u[0,0] >= 0.)
    m.addConstr(1.- u[0,0] - u[0,4] >= 0.)
    m.addConstr(1.- u[0,2] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]-min_v+u[0,2] >= 0.)
    m.addConstr(1.- u[0,1] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]+1.0+u[0,1] >= 0.)
    m.addConstr(1.- u[0,3] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]+1.0+u[0,3] >= 0.)
    m.setObjective(-u[0, 4]+ regCoefficient * (u[0,0] ** 2 + u[0,1] ** 2 + u[0,2] ** 2+ u[0,3] ** 2), GRB.MINIMIZE)
    m.optimize()
    modifiedU = u.getAttr("x")[0,0:4]
    cbfValue = dhdx[0]*(modifiedU[0]*onp.cos(x[2])) + dhdx[1]*(modifiedU[0]*onp.sin(x[2])) + dhdx[2]*modifiedU[1] + dhdx[3]*(modifiedU[2]*onp.cos(x[5])) + dhdx[4]*(modifiedU[2]*onp.sin(x[5])) + dhdx[5]*modifiedU[3] + hx**3
    '''
    if cbfValue < 0:
        raise ValueError("CBF value is not positive when calculating center!")
    '''
    return torch.tensor(modifiedU), cbfF, cbfg

# gauge map
def gaugeMapPred(v, F, g, uCenter):
   maxv = torch.max(abs(v))
   result = torch.tensor(maxv / (gaugeFunctionPred(v, F, g, uCenter)) * v)
   #if (result + uCenter) @ F > g:
      #raise ValueError("CBF violation!")
   return result + uCenter

def gaugeFunctionPred(v, F, g, uCenter):
   v = v.unsqueeze(0).float()
   vF = torch.sum(v @ F.float(), 0)
   uCenter = uCenter.unsqueeze(0)
   gSub = (uCenter @ F).item()
   result = vF / (g - gSub)
   for i in range(len(inputBoundsMatrix)):
      inputConstraint = v @ inputBoundsMatrix[i]
      constraint = inputConstraint / (inputBounds[i] - uCenter.float() @ inputBoundsMatrix[i])
      result = torch.max(constraint, result)
   return result 

# gauge map for the whole training dataset
# the only difference with above gaugeMapPred is the dimension
def gaugeMapAll(v, F, g, uCenter):
   maxv = torch.max(abs(v), 1)[0].unsqueeze(1)
   result = maxv / (gaugeFunctionAll(v, F, g, uCenter)) * v
   return result + uCenter

def gaugeFunctionAll(v, F, g, uCenter):
   vF = torch.sum(v * F, 1).unsqueeze(1)
   gSub = torch.sum(uCenter.float() * F, 1).unsqueeze(1)
   result = vF / (g - gSub)
   for i in range(len(inputBoundsMatrix)):
      inputConstraint = v @ inputBoundsMatrix[i]
      constraint = inputConstraint / (inputBounds[i] - uCenter.float() @ inputBoundsMatrix[i])
      result = torch.max(constraint, result)
   return result 

# Solve the CBF-QP problem using Gurobi
def CBF_QP_Gurobi(x, u_opt):
    x  = onp.array(x) # Current states
    #QP_start_time = time.time()
    m = gp.Model("QP")
    m.setParam('OutputFlag', 0)
    m.params.NonConvex = 2
    u = m.addMVar(shape=(1,4), lb=u_lb, ub=u_ub, name='u') # Variable control inputs
    hx   = ZCBF(x) # CBF value
    dhdx = num_grad(x) # CBF gradient value
    m.addConstr(dhdx[0]*(u[:,0]*onp.cos(x[2])) + dhdx[1]*(u[:,0]*onp.sin(x[2])) + dhdx[2]*u[:,1] + \
          dhdx[3]*(u[:,2]*onp.cos(x[5])) + dhdx[4]*(u[:,2]*onp.sin(x[5])) + dhdx[5]*u[:,3] + hx**3>= 0) # CBF constraint
    V = (u[0,0]-u_opt[0])*(u[0,0]-u_opt[0])+(u[0,1]-u_opt[1])*(u[0,1]-u_opt[1])+(u[0,2]-u_opt[2])*(u[0,2]-u_opt[2])+(u[0,3]-u_opt[3])*(u[0,3]-u_opt[3]) # Objective function
    m.setObjective(V, GRB.MINIMIZE)
    m.optimize()
    modifiedU = u.getAttr("x").tolist()[0]
    return modifiedU

# solve the nonlinear MPC using CasADi
def CFTOC_NMPC(x_init, x_goal, T, N):
    nx = 6
    nu = 4
    x1 = MX.sym('x1') # Declare model variables
    x2 = MX.sym('x2')
    x3 = MX.sym('x3')
    x4 = MX.sym('x4')
    x5 = MX.sym('x5')
    x6 = MX.sym('x6')
    u1 = MX.sym('u1')
    u2 = MX.sym('u2')
    u3 = MX.sym('u3')
    u4 = MX.sym('u4')

    x  = vertcat(x1, x2, x3, x4, x5, x6)
    u  = vertcat(u1, u2, u3, u4)
    xdot = vertcat( u1*cos(x3), u1*sin(x3), u2, u3*cos(x6), u3*sin(x6), u4 ) # Model equations

    L = 5*(x1-x_goal[0])**2 + 2*(x2-x_goal[1])**2 + angle_cost_coefficient*(x3-x_goal[2])**2 + \
          5*(x4-x_goal[3])**2 + 2*(x5-x_goal[4])**2 + angle_cost_coefficient*(x6-x_goal[5])**2 + \
          (1*u1**2 + 0.1*u2**2 + 1*u3**2 + 0.1*u4**2) * input_cost_coefficient # Objective term
    M = 4 # RK4 steps per interval
    DT = T/N/M
    f = Function('f', [x, u], [xdot, L])
    X0 = MX.sym('X0', nx)
    U  = MX.sym('U',  nu)
    X = X0
    Q = 0
    for j in range(M):
        k1, k1_q = f(X, U)
        k2, k2_q = f(X + DT/2 * k1, U)
        k3, k3_q = f(X + DT/2 * k2, U)
        k4, k4_q = f(X + DT * k3, U)
        X=X+DT/6*(k1 +2*k2 +2*k3 +k4)
        Q = Q + DT/6*(k1_q + 2*k2_q + 2*k3_q + k4_q)
    F = Function('F', [X0, U], [X, Q],['x0','p'],['xf','qf'])
    # Start with an empty NLP
    w  =[]
    w0 = []
    lbw = []
    ubw = []
    J = 0
    lbg = []
    ubg = []
    g = []
    Xk = MX.sym('X0', nx)
    w += [Xk]
    lbw += x_init
    ubw += x_init
    w0  += x_init
    hx   = ZCBF(np.array(x_init))
    dhdx = num_grad(np.array(x_init))
    cbfValue = [ dhdx[0]*(u1*onp.cos(x_init[2])) + dhdx[1]*(u1*onp.sin(x_init[2])) + dhdx[2]*u2 + \
          dhdx[3]*(u3*onp.cos(x_init[5])) + dhdx[4]*(u3*onp.sin(x_init[5])) + dhdx[5]*u4 + hx**3 ] # LHS of the CBF constraint
    # Formulate the NLP
    for k in range(N): # N is the MPC horizon steps
        Uk = MX.sym('U_' + str(k), nu) # New NLP variable for the control
        w   += [Uk]
        lbw += u_lb
        ubw += u_ub
        w0  += [0,0,0,0]
        Fk = F(x0=Xk, p=Uk) # Integrate till the end of the interval
        Xk_end = Fk['xf']
        J=J+Fk['qf']
        Xk = MX.sym('X_' + str(k+1), nx) # New NLP variable for state at end of interval\
        
        w   += [Xk]
        lbw += [-inf,-inf,-inf,-inf,-inf,-inf]
        ubw += [ inf, inf, inf, inf, inf, inf]
        w0  += [0,0,0,0,0,0]
        g   += [Xk_end-Xk] # Add equality constraint
        lbg += [0,0,0,0,0,0]
        ubg += [0,0,0,0,0,0]
    prob = {'f': J, 'x': vertcat(*w), 'g': vertcat(*g)} # Create an NLP solver
    opts = {'ipopt.print_level':0, 'print_time':0}
    solver = nlpsol('solver', 'ipopt', prob, opts);
    sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg) # Solve the NLP
    w_opt = sol['x'].full().flatten()
    u_opt = list(w_opt[6:10])
    return u_opt

def trajCost(x, u):
    """
    calculate the trajectory cost for the given states and inputs

    Args:
        x:
            system trajectory states
        
        u:
            system trajectory inputs

    Returns:
        cost
    """

    cost = 0
    x_goal = [-5.0, 0.0, goal_angle, 5.0, 0.0, 0.0] # goal state
    for i in range(u.shape[0]):
      cost += (5*(x[i,0]-x_goal[0])**2 + 2*(x[i,1]-x_goal[1])**2 + angle_cost_coefficient*(x[i,2]-x_goal[2])**2 + \
          5*(x[i,3]-x_goal[3])**2 + 2*(x[i,4]-x_goal[4])**2 + angle_cost_coefficient*(x[i,5]-x_goal[5])**2 + \
          (1*u[i,0]**2 + 0.1*u[i,1]**2 + 1*u[i,2]**2 + 0.1*u[i,3]**2) * input_cost_coefficient) * 0.1
    return cost

def distValuePlot(x):
    """
    plot the distance values between two agents

    Args:
        x:
            system trajectory states

    Returns:
        no return
    """
    distValue = ((x[:,0] - x[:,3]) ** 2 + (x[:,1] - x[:,4]) ** 2) ** (1/2)
    figurex = np.linspace(0, x.shape[0], num=x.shape[0])
    leadVelocityLine = np.linspace(Ds, Ds, num=x.shape[0])
    print("minimal distance is: ", min(distValue))
    plt.plot(figurex, distValue)
    plt.plot(figurex, leadVelocityLine, c = 'r')
    plt.xlabel('time')
    plt.ylabel("Distance between two aricrafts")
    #images_dir = '/content/drive/My Drive/Colab Notebooks/figures'
    #plt.savefig(f"{images_dir}/cbfValue-NN35.png")


def readdata():
    dir = '/content/drive/My Drive/Colab Notebooks'
    #plt.savefig(f"{dir}/QP-CBFvalue.png")
    with open('ACAdata.npy', 'rb') as f:
      dataSum = np.load(f)
    print(dataSum.shape)
    return dataSum

def safe_closed_loop_nn(x_init, x_goal, N_sim, plot, nnController, safety, gauge):

    """
    Closed-loop Simulations with NN controller

    Args:
        x_init:
            initial state of system

        x_goal:
            goal state of system
          
        N_sim:
            simulation steps number
        
        plot (bool):
            plot the distance values and trajectories or not
        
        nnController:
            trained NN controller
        
        safety (bool):
            safety = 1 --> NN-CBF-Gauge or NN-CBF-QP
            safety = 0 --> NN

        gauge (bool):
            gauge = 1 --> NN-CBF-Gauge
            gauge = 0 --> NN or NN-CBF-QP

    Returns:
        the simulated closed-loop system states list, the simulated closed-loop system inputs list
    
    Remark:
    # if gauge = 1 and safety = 1 --> NN-CBF-Gauge
    # if gauge =0 and safety = 1  --> NN-CBF-QP
    # if gauge = 0 and safety =0  --> NN
    """

    Xclose = []
    Uclose = []
    Xclose.append(onp.array(x_init))
    xt = x_init

    for t in range(N_sim):
      u_opt = nnController(torch.tensor([xt]).float())
      u_opt = u_opt.tolist()[0]
      if safety and gauge == 0: # NN-CBF-QP
        #xt = xt.tolist()[0]
        u_mod = CBF_QP_Gurobi(xt, u_opt) # Solve the CBF-QP problem at the current time
      elif safety and gauge: # NN-CBF-Gauge
        u_center, cbfF, cbfg = chebyshevGurobi(xt) # get the Chebyshev center at the current time
        u_mod = gaugeMapPred(torch.tensor(u_opt), cbfF, cbfg, torch.tensor(u_center))[0] # gauge map
      else: # NN
        u_mod = u_opt
      xt = dynamics_RK4(xt, u_mod, T, N) # Evolve the system
      #xt = my_dynamics(xt, u_mod).tolist()
      #print(xt, xt.shape)
      Xclose.append(onp.array(xt))
      Uclose.append(onp.array(u_mod))

    Xclose = onp.array(Xclose)
    Uclose = onp.array(Uclose)
    cost = trajCost(Xclose, Uclose) # calculate the trajectory cost
    print("the trajectory cost is: ", cost)
    print("-------------------------------")
    if plot:
      distValuePlot(Xclose)
      tgrid = [ts*k for k in range(N_sim)]
      plt.figure(figsize=(10,5))
      plt.clf()
      plt.plot(Xclose[:,0], Xclose[:,1], '-')
      plt.plot(Xclose[:,3], Xclose[:,4], '-')
      plt.xlabel('$p_x$')
      plt.ylabel('$p_y$')
      plt.legend(['agent a','agent b'])
      plt.xlim([-3,3])
      plt.ylim([-2,2])
      plt.axis('equal')
      plt.grid()
      plt.figure()
      plt.clf()
      plt.step(tgrid, Uclose[:,0], '-.')
      plt.step(tgrid, Uclose[:,1], '-.')
      plt.step(tgrid, Uclose[:,2], '-.')
      plt.step(tgrid, Uclose[:,3], '-.')
      plt.xlabel('t')
      plt.legend(['u1','u2','u3','u4'])
      plt.grid()
      plt.show()
    return Xclose, Uclose



## The following functions are used in the constructive CBF [Squires et al]

def b(px,theta):
    return px - (v/w)*onp.sin(theta)

def c(py,theta):
    return py + (v/w)*onp.cos(theta)

def db(x):
    return b(x[0],x[2]) - b(x[3],x[5])

def dc(x):
    return c(x[1],x[2]) - c(x[4],x[5])

def db_grad(x):
    return x[:, 0] - (v/w)*torch.sin(x[:, 2]) - x[:, 3] + (v/w)*torch.sin(x[:, 5])

def dc_grad(x):
    return x[:, 1] + (v/w)*torch.cos(x[:, 2]) - x[:, 4] - (v/w)*torch.cos(x[:, 5])

def compute_A2(x):
    A21, theta21 = phasor_add(-2*db(x)*(v/w), 2*db(x)*(v/w), x[2]+onp.pi/2, x[5]+onp.pi/2)
    A22, theta22 = phasor_add(-2*dc(x)*(v/w), 2*dc(x)*(v/w), x[2], x[5])
    A2,  theta2  = phasor_add(A21, A22, theta21, theta22)
    return A2

def phasor_add(A1,A2,theta1,theta2):
# Compute the phasor addition of two cosines
    A3 = onp.sqrt( (A1*onp.cos(theta1)+A2*onp.cos(theta2))**2 + (A1*onp.sin(theta1)+A2*onp.sin(theta2))**2 )
    if A1*onp.cos(theta1)+A2*onp.cos(theta2)==0:
      theta3 = onp.sign(A1*onp.sin(theta1)+A2*onp.sin(theta2))*pi/2
    elif A1*onp.cos(theta1)+A2*onp.cos(theta2)>0:
      theta3 = onp.arctan((A1*onp.sin(theta1)+A2*onp.sin(theta2))/(A1*onp.cos(theta1)+A2*onp.cos(theta2)))
    else:
      theta3 = onp.arctan((A1*onp.sin(theta1)+A2*onp.sin(theta2))/(A1*onp.cos(theta1)+A2*onp.cos(theta2))) + onp.pi 
    return A3, theta3

def compute_A2_grad(x):
    A21, theta21 = phasor_add_grad(-2*db_grad(x)*(v/w), 2*db_grad(x)*(v/w), x[:, 2]+torch.pi/2, x[:, 5]+torch.pi/2)
    A22, theta22 = phasor_add_grad(-2*dc_grad(x)*(v/w), 2*dc_grad(x)*(v/w), x[:, 2], x[:, 5])
    A2,  theta2  = phasor_add_grad(A21, A22, theta21, theta22)
    return A2

def phasor_add_grad(A1,A2,theta1,theta2):
# Compute the phasor addition of two cosines
    A3 = torch.sqrt( (A1*torch.cos(theta1)+A2*torch.cos(theta2))**2 + (A1*torch.sin(theta1)+A2*torch.sin(theta2))**2 )
    if A1*torch.cos(theta1)+A2*torch.cos(theta2)==0:
      theta3 = torch.sign(A1*torch.sin(theta1)+A2*torch.sin(theta2))*pi/2
    elif A1*torch.cos(theta1)+A2*torch.cos(theta2)>0:
      theta3 = torch.arctan((A1*torch.sin(theta1)+A2*torch.sin(theta2))/(A1*torch.cos(theta1)+A2*torch.cos(theta2)))
    else:
      theta3 = torch.arctan((A1*torch.sin(theta1)+A2*torch.sin(theta2))/(A1*torch.cos(theta1)+A2*torch.cos(theta2))) + torch.pi 
    return A3, theta3

In [ ]:
# NN architecture for DPC
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(6, 50),
            nn.Sigmoid(),
            nn.Linear(50, 200),
            nn.Sigmoid(),
            nn.Linear(200, 50),
            nn.Sigmoid(),
            nn.Linear(50, 4),
            nn.Tanh()
        )

    def forward(self, x):
        next_u = self.stack(x)
        return next_u


In [ ]:
## generate training data using MPC

import time

# starting points for generating trajectories
X_start = [[0.5, 0.0, start_angle, -0.5, -0.0, 0.0]]
#X_start = [[0.5, 0.0, start_angle, -0.5, -0.0, 0.0]]
x_goal = [-5.0, 0.0, goal_angle, 5.0, 0.0, 0.0] # goal for the system
Xcl_total = [] # store the closed-loop trajectories
Ucl_total = [] # store the closed-loop inputs

MPC_start_time = time.time()
for i in range(len(X_start)):
  x_init = X_start[i]
  Xcl, Ucl = safe_closed_loop(x_init, x_goal, N_sim=200, plot=1)
  for i in range(Ucl.shape[0]):
    Xcl_total.append(Xcl[i,:])
    Ucl_total.append(Ucl[i,:])
total_time = time.time() - MPC_start_time

'''
images_dir = '/content/drive/My Drive/Colab Notebooks/figures'
with open(f"{images_dir}/mpcAngle.txt", 'w') as f:
  f.write(str(Xcl[:,2].tolist()))
  f.write("-----------------w_{a} above")
  f.write(str(Xcl[:,5].tolist()))
'''

print("----------------")
print("MPC total running time for %s starting point: " % len(X_start), total_time)
print("MPC average time: ", total_time / len(X_start))

In [ ]:
def ODE_RHS_grad(x,u):
# RHS of the aircraft dynamics
    xdot = (u[:,0]*torch.cos(x[:,2])).reshape((200, 1))
    xdot = torch.cat((xdot, (u[:,0]*torch.sin(x[:,2])).reshape((200, 1))), 1)
    xdot = torch.cat((xdot, (u[:,1]).reshape((200, 1))), 1)
    xdot = torch.cat((xdot, (u[:,2]*torch.cos(x[:,5])).reshape((200, 1))), 1)
    xdot = torch.cat((xdot, (u[:,2]*torch.sin(x[:,5])).reshape((200, 1))), 1)
    xdot = torch.cat((xdot, (u[:,3]).reshape((200, 1))), 1)
    #xdot = torch.tensor([u[:,0]*torch.cos(x[:,2]), u[:,0]*torch.sin(x[:,2]), u[:,1], u[:,2]*torch.cos(x[:,5]), u[:,2]*torch.sin(x[:,5]), u[:,3] ])
    return xdot

def dynamics_RK4_grad(x, u, T, N):
# RK4 integrator
    M = 4 # RK4 steps per interval
    DT = T/N/M
    X = x
    U = u
    for j in range(M):
        #print("x: ", X, X.shape) # (1,6)
        #print("u: ", U, U.shape) # (1,4)
        k1 = ODE_RHS_grad(X, U)
        #print(type(k1), k1.shape, DT/2)
        k2 = ODE_RHS_grad(X + DT/2 * k1, U)
        k3 = ODE_RHS_grad(X + DT/2 * k2, U)
        k4 = ODE_RHS_grad(X + DT * k3, U)
        X=X+DT/6*(k1 +2*k2 +2*k3 +k4)
    return X

def mpcCost(x, u):
    criterion = torch.nn.MSELoss()
    x_goal = torch.Tensor([-5.0, 0.0, goal_angle, 5.0, 0.0, 0.0]) # goal state
    input_goal = torch.Tensor([0.0, 0.0, 0.0, 0.0])
    cost = (5 * criterion(x[0, 0], x_goal[0]).float() + 2*criterion(x[0, 1], x_goal[1]).float() + angle_cost_coefficient*criterion(x[0, 2], x_goal[2]).float() + \
          5*criterion(x[0, 3], x_goal[3]).float() + 2*criterion(x[0, 4], x_goal[4]).float() + angle_cost_coefficient*criterion(x[0, 5], x_goal[5]).float() + \
          (1*criterion(u[0, 0], input_goal[0]).float() + 0.1*criterion(u[0, 1], input_goal[1]).float() + 1*criterion(u[0, 2], input_goal[2]).float() + 0.1*criterion(u[0, 3], input_goal[3])).float() * input_cost_coefficient)
    return cost

def DiffPredictiveControlLoss(x, NNoutput):
    lossMPC = mpcCost(x, NNoutput) * 10
    stateLoss = 0.0
    distance = (x[0, 0] - x[0, 3]) ** 2 + (x[0, 1] - x[0, 4]) ** 2
    if distance <= 0.25:
        stateLoss += (0.25 - distance) * 10
    inputLoss = 0.0
    if NNoutput[0, 0] < 0.1:
        inputLoss += (0.1 - NNoutput[0, 0]) * 10
    if NNoutput[0, 2] < 0.1:
        inputLoss += (0.1 - NNoutput[0, 2]) * 10
    return lossMPC + stateLoss + inputLoss

In [ ]:
def mpcCostBatch(x, u):
    criterion = torch.nn.MSELoss()
    x_goal = torch.Tensor([-5.0, 0.0, goal_angle, 5.0, 0.0, 0.0]) # goal state
    input_goal = torch.Tensor([0.0, 0.0, 0.0, 0.0])
    cost = ((5 *(x[:, 0] - x_goal[0]) ** 2 + 2*(x[:, 1]-x_goal[1])**2 + angle_cost_coefficient*(x[:, 2]-x_goal[2])**2 + \
          5*(x[:, 3]-x_goal[3])**2 + 2*(x[:, 4]-x_goal[4])**2 + angle_cost_coefficient*(x[:, 5]-x_goal[5])**2) * 1000 + \
          (2*(u[:, 0]-input_goal[0])**2 + 0.1*(u[:, 1]-input_goal[1])**2 + 2*(u[:, 2]-input_goal[2])**2 + 0.1*(u[:, 3]-input_goal[3]))**2)* 0.0005#input_cost_coefficient
    return cost

def DiffPredictiveControlLossBatch(x, NNoutput):
  
    #print("x: ", x, x.shape)
    #print("NNoutput: ", NNoutput, NNoutput.shape)
    #print("mpcCostBatch(x, NNoutput): ", mpcCostBatch(x, NNoutput), mpcCostBatch(x, NNoutput).shape)
    lossMPC = torch.sum(mpcCostBatch(x, NNoutput))
    #print("lossMPC: ", lossMPC, lossMPC.shape)
    #print("--------------------")
    stateLoss = 0.0
    distance = ((x[:, 0] - x[:, 3]) ** 2 + (x[:, 1] - x[:, 4]) ** 2 - 0.25)
    distance = torch.where(distance < 0.0, distance, 0.)
    stateLoss += torch.sum( - distance * 100)
    inputLoss = 0.0
    NNoutput[:, 0] = torch.where(NNoutput[:, 0] < 0.1, NNoutput[:, 0], 0.1)
    NNoutput[:, 2] = torch.where(NNoutput[:, 2] < 0.1, NNoutput[:, 2], 0.1)
    inputLoss += torch.sum( (0.1 - NNoutput[:, 0]) * 100)
    inputLoss += torch.sum( (0.1 - NNoutput[:, 2]) * 100)
    return lossMPC + stateLoss + inputLoss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks

In [ ]:
# train NN-Diff-CBF-QP

def trainNNcontroller(X_train):

    # data pre-process
    dataNum = X_train.shape[0]
    print("training data num: ", dataNum)
    trainF = torch.rand(dataNum, 4) # CBF constraint for the whold training dataset: trainF * u <= trainG
    trainG = torch.rand(dataNum)
    for i in range(dataNum):
        x = X_train[i,:].reshape((1, 6))
        hx   = ZCBF_grad(x)
        dhdx = num_grad_grad(x)
        #print(dhdx)
        cbfF = (- dhdx[0]*torch.cos(x[0, 2]) - dhdx[1]*torch.sin(x[0, 2])).reshape((1,1))
        cbfF = torch.cat((cbfF, (- dhdx[2]).reshape((1,1))), 1)
        cbfF = torch.cat((cbfF, (- dhdx[3]*torch.cos(x[0, 5]) - dhdx[4]*torch.sin(x[0, 5])).reshape((1,1))), 1)
        trainF[i,:] = torch.cat((cbfF, (- dhdx[5]).reshape((1,1))), 1)
        cbfg = torch.tensor(hx ** 3)
        trainG[i] = cbfg.item()

    # NN setup
    model = NeuralNetwork()
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learningRate)

    # Differentiabl QP setup
    u_dim = 4 # variable dim (control inputs dim is 4)
    u_mod = cp.Variable(u_dim+1)
    u_init = cp.Parameter(u_dim)
    cbf_f = cp.Parameter(u_dim)
    cbf_g = cp.Parameter(1)
    unsafe_penalty = 1000.0
    constraints = [u_mod[4] >= 0.0, u_mod[0:4] >= u_lb, u_mod[0:4] <= u_ub, cbf_f @ u_mod[0:4] <= cbf_g + u_mod[4]]
    objective = cp.Minimize(cp.pnorm(u_mod[0:4] - u_init, p=2) + u_mod[4] * unsafe_penalty)
    problem = cp.Problem(objective, constraints)
    cvxpylayer = CvxpyLayer(problem, parameters=[u_init, cbf_f, cbf_g], variables=[u_mod])

    # Gradient Descent
    for epoch in range(200):
      y_pred = model(X_train) # Forward pass
      loss_epoch = 0
      # cvxpylayer does not support batch computation, so we must do it one by one
      loss = 0
      currentX = X_train
      for horizon in range(50):
          next_u = model(currentX)
          for each_data_index in range(trainF.shape[0]):
            solution, = cvxpylayer(next_u[each_data_index, :], trainF[each_data_index].unsqueeze(0), trainG[each_data_index].unsqueeze(0))
            if each_data_index == 0:
                next_u_mod = solution
            else:
                next_u_mod = torch.cat((next_u_mod, solution), 0)
            subLoss = DiffPredictiveControlLoss(currentX[each_data_index, :].reshape((1,6)), solution)
            loss += subLoss
          currentX = dynamics_RK4_grad(currentX, next_u_mod, T, 10) #next_u_mod
          #print("next x:", currentX)
          trainF = torch.rand(dataNum, 4) # CBF constraint for the whold training dataset: trainF * u <= trainG
          trainG = torch.rand(dataNum)
          hx   = ZCBF_grad(currentX)
          dhdx = num_grad_grad(currentX)
          for x_index in range(currentX.shape[0]):
            x = currentX[x_index, :].reshape((1, 6))
            hx   = ZCBF_grad(x)
            dhdx = num_grad_grad(x)
            cbfF = (- dhdx[0]*torch.cos(x[0, 2]) - dhdx[1]*torch.sin(x[0, 2])).reshape((1,1))
            cbfF = torch.cat((cbfF, (- dhdx[2]).reshape((1,1))), 1)
            cbfF = torch.cat((cbfF, (- dhdx[3]*torch.cos(x[0, 5]) - dhdx[4]*torch.sin(x[0, 5])).reshape((1,1))), 1)
            trainF[i,:] = torch.cat((cbfF, (- dhdx[5]).reshape((1,1))), 1)
            cbfg = hx ** 3
            trainG[i] = cbfg
      loss /= dataNum
      print('epoch: ', epoch,' loss: ', loss)
      out = optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    return model

X_train = torch.Tensor(readdata()).float()

# collect CBF values
cbfValue = []
for i in range(X_train.shape[0]):
  newValue = ZCBF(X_train[i,:])
  cbfValue.append(newValue)

NNcontrollerDiff = trainNNcontroller(X_train) # get the trained NN-Diff-CBF-QP
print("minimal CBF value: ", min(cbfValue))
print("------------------------------------")

In [ ]:
'''
test NN-Diff-CBF-QP
'''
#NNcontrollerDiff = NeuralNetwork()
#NNcontrollerDiff.load_state_dict(torch.load("models/Aircraft/horizon5diffQP"))
safeIndex = 1
gaugeIndex = 0
X_start = [1.1, 0.0, 3.141592653589793, -1.1, -0.0, 0.0]
x_goal = [-5.0, 0.0, 3.141592653589793, 5.0, 0.0, 0.0]
test_start_time = time.time()
controller = NNcontrollerDiff#NNcontrollerDiff
newXcl, newUcl = safe_closed_loop_nn(x_init, x_goal, N_sim=200, plot=1, nnController=controller, safety = safeIndex, gauge=gaugeIndex)


In [ ]:
def mpcCostBatch(x, u):
    criterion = torch.nn.MSELoss()
    x_goal = torch.Tensor([-5.0, 0.0, goal_angle, 5.0, 0.0, 0.0]) # goal state
    input_goal = torch.Tensor([0.0, 0.0, 0.0, 0.0])
    cost = ((5 *(x[:, 0] - x_goal[0]) ** 2 + 2*(x[:, 1]-x_goal[1])**2 + 1.0 *(x[:, 2]-x_goal[2])**2 + \
          5*(x[:, 3]-x_goal[3])**2 + 2*(x[:, 4]-x_goal[4])**2 + 1.0 *(x[:, 5]-x_goal[5])**2) + \
          (0.01*(u[:, 0]-input_goal[0])**2 + 0.1*(u[:, 1]-input_goal[1])**2 + 0.01*(u[:, 2]-input_goal[2])**2 + 0.1*(u[:, 3]-input_goal[3]))**2)* 1.0#input_cost_coefficient
    return cost

def DiffPredictiveControlLossBatch(x, NNoutput):
  
    #print("x: ", x, x.shape)
    #print("NNoutput: ", NNoutput, NNoutput.shape)
    #print("mpcCostBatch(x, NNoutput): ", mpcCostBatch(x, NNoutput), mpcCostBatch(x, NNoutput).shape)
    lossMPC = torch.sum(mpcCostBatch(x, NNoutput))
    #print("lossMPC: ", lossMPC, lossMPC.shape)
    #print("--------------------")
    stateLoss = 0.0
    distance = ((x[:, 0] - x[:, 3]) ** 2 + (x[:, 1] - x[:, 4]) ** 2 - 0.25)
    distance = torch.where(distance < 0.0, distance, 0.)
    stateLoss += torch.sum( - distance * 100)
    inputLoss = 0.0
    NNoutput[:, 0] = torch.where(NNoutput[:, 0] < 0.1, NNoutput[:, 0], 0.1)
    NNoutput[:, 2] = torch.where(NNoutput[:, 2] < 0.1, NNoutput[:, 2], 0.1)
    inputLoss += torch.sum( (0.1 - NNoutput[:, 0]) * 100)
    inputLoss += torch.sum( (0.1 - NNoutput[:, 2]) * 100)
    return lossMPC + stateLoss + inputLoss

In [ ]:
predictiveHorizon = 100


u_r_lb = torch.tensor([min_v, -1.0, min_v, -1.0, 0, 0])
u_r_ub = torch.tensor([1.0,  1.0, 1.0,  1.0, 10, 100])

def chebyshevGurobi(x):
    #start_time = time.time()
    m = gp.Model("LP")
    m.setParam('OutputFlag', 0)
    u = m.addMVar(shape=(1,6), lb=u_r_lb, ub=u_r_ub, name='u')
    unsafe_penalty = 1000.0
    regular_unique = 0.001
    hx   = ZCBF_grad(x)
    dhdx = num_grad_grad(x)
    cbfF = (- dhdx[0]*torch.cos(x[0, 2]) - dhdx[1]*torch.sin(x[0, 2])).reshape((1,1))
    cbfF = torch.cat((cbfF, (- dhdx[2]).reshape((1,1))), 0)
    cbfF = torch.cat((cbfF, (- dhdx[3]*torch.cos(x[0, 5]) - dhdx[4]*torch.sin(x[0, 5])).reshape((1,1))), 0)
    cbfF = torch.cat((cbfF, (- dhdx[5]).reshape((1,1))), 0)
    cbfg = hx ** 3
    #chebyPara = (dhdx[0]*onp.cos(x[2]) - dhdx[1]*onp.sin(x[2]))**2 + dhdx[2]**2 + (dhdx[3]*onp.cos(x[5]) - dhdx[4]*onp.sin(x[5]))**2 + dhdx[5] ** 2
    chebyPara = (dhdx[0]*torch.cos(x[0, 2])) ** 2 + (dhdx[1]*torch.sin(x[0, 2]))**2 + dhdx[2]**2 + (dhdx[3]*torch.cos(x[0, 5])) ** 2 + (dhdx[4]*torch.sin(x[0, 5]))**2 + dhdx[5] ** 2
    m.addConstr(dhdx[0]*(u[0,0]*torch.cos(x[0, 2])) + dhdx[1]*(u[0,0]*torch.sin(x[0, 2])) + dhdx[2]*u[0,1] + dhdx[3]*(u[0,2]*torch.cos(x[0, 5])) + dhdx[4]*(u[0,2]*torch.sin(x[0, 5])) + dhdx[5]*u[0,3] + hx**3 - u[0,4] * chebyPara >= 0)

    m.addConstr(-u[0,4]-min_v+u[0,0] >= 0.)
    m.addConstr(1.- u[0,0] - u[0,4] >= 0.)
    m.addConstr(1.- u[0,2] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]-min_v+u[0,2] >= 0.)
    m.addConstr(1.- u[0,1] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]+1.0+u[0,1] >= 0.)
    m.addConstr(1.- u[0,3] - u[0,4] >= 0.)
    m.addConstr(-u[0,4]+1.0+u[0,3] >= 0.)
    m.setObjective(-u[0, 4] + regular_unique * (u[0,0] ** 2 + u[0,1] ** 2 + u[0,2] ** 2+u[0,3] ** 2), GRB.MINIMIZE)# + unsafe_penalty * u[0, 5]
    m.optimize()
    center = u.getAttr("x")[0,0:4]
    #print("center & cbfF & cbfg & chebypara & radius", center, cbfF, cbfg, chebyPara, u.getAttr("x")[0,4])
    #print("---------------------------------------------")
    #cbfValue = dhdx[0]*(modifiedU[0]*onp.cos(x[2])) + dhdx[1]*(modifiedU[0]*onp.sin(x[2])) + dhdx[2]*modifiedU[1] + dhdx[3]*(modifiedU[2]*onp.cos(x[5])) + dhdx[4]*(modifiedU[2]*onp.sin(x[5])) + dhdx[5]*modifiedU[3] + hx**3
    return torch.tensor(center), cbfF, cbfg


In [ ]:
# Gradient Descent
train_start_time = time.time()
for epoch in range(10): #epochNumber
    currentX = X_train 
    loss = 0
    for horizon in range(predictiveHorizon):
        next_u = model(currentX.float())
        trainF = torch.rand(X_train.shape[0], 4) # CBF constraint for the whold training dataset: trainF * u <= trainG
        trainG = torch.rand(X_train.shape[0])
        for each_data_index in range(dataNum):
            center, cbf_f, cbf_g = chebyshevGurobi(currentX[each_data_index, :].reshape((1, 6)))
            if each_data_index == 0:
                u_center_list = center.reshape((1,4))
                trainF = torch.transpose(cbf_f, 0, 1)
                trainG = cbf_g.reshape((1,1))
            else:
                u_center_list = torch.cat((u_center_list, center.reshape((1,4))), 0)
                trainF = torch.cat((trainF, torch.transpose(cbf_f, 0, 1)), 0)
                trainG = torch.cat((trainG, cbf_g.reshape((1,1))), 0)
        next_u = gaugeMapAll(next_u, trainF, trainG, u_center_list) # gauge map
        subLoss = DiffPredictiveControlLossBatch(currentX, next_u)
        loss += subLoss
        currentX = dynamics_RK4_grad(currentX, next_u, T, 10)
    loss /= X_train.shape[0]
    print('epoch: ', epoch,' loss: ', loss.item())
    out = optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#NNcontrollerNondiff = model
print("----------------------------")
print("training time cost is: ", time.time()-train_start_time)

In [ ]:
safeIndex = 1
gaugeIndex = 1
X_start = [1.3, 0.0, 3.141592653589793, -1.3, -0.0, 0.0]
x_goal = [-5.0, 0.0, 3.141592653589793, 5.0, 0.0, 0.0]
x_init = X_start
test_start_time = time.time()
controller = model
newXcl, newUcl = safe_closed_loop_nn(x_init, x_goal, N_sim=200, plot=1, nnController=controller, safety = safeIndex, gauge=gaugeIndex)
